# Workshop: `teradataml` & Descriptive Statistics

In [ ]:
!pip install teradataml --upgrade

In [ ]:
try:
    import tdprepview
except ImportError:
    !pip install tdprepview
    import tdprepview

# Database connection and synthetic data upload

## Database Connection

This code cell imports the teradataml package, prints its version, and creates a context for connecting to a Teradata database.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
import teradataml as tdml
print(tdml.__version__)
tdml.create_context(host="___", username="demo_user", password="___")

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To create a context, you need to provide the server URL, username, and the database name. These are required to establish a connection to the Teradata environment.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
import teradataml as tdml
print(tdml.__version__)
tdml.create_context("see at the clearscape dashboard",
                    "demo_user", "only you know :)")
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## Data Upload

This code cell runs a notebook to generate data tables, uploads the tables to the Teradata database, and checks the data types of the tables.

In [ ]:
%run tables-ddls-banking.ipynb #  generates the data

my_tables_dict = {
        "Customer_Dim": customer_dim,
        "Account_Dim": account_dim,
        "Account_Customer_Map": account_customer_map,
        "Transaction_Fact": transaction_fact,
        "Balance_Fact": balance_fact,
        "Interaction_Fact": interaction_fact,
        "Master_Table": master_table,
        "Customer_Details":customer_details
    }

#upload
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
for tablename, table in my_tables_dict.items():
    tdml.copy_to_sql(table, tablename,if_exists="replace",primary_index= table.columns[0])
    print(tablename, " uploaded")

# check tdtypes
types_dict = {}
for tablename in my_tables_dict.keys():
    types_dict[tablename] = tdml.DataFrame(tablename).tdtypes._column_names_and_types

## Function for pretty printing SQLs

This code cell defines a function `prettyprint_sql` that formats SQL queries for better readability.

In [ ]:
import sqlparse
def prettyprint_sql(query):
    print(sqlparse.format(
            query,
            reindent=True, 
            keyword_case='upper'
    ))

## Inspect Data

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def display_dataframes_in_tabs(table_names):
    # Temporarily set pandas display options to ensure all columns are visible
    with pd.option_context('display.max_columns', None):
        # Create a list to hold each tab's contents and their titles
        tab_contents = []
        tab_titles = table_names
        
        for table_name in table_names:
            # Get the first ten rows of the DataFrame
            df_head = tdml.DataFrame(table_name)
            table_output = widgets.Output()  # Create an output widget for the table
            with table_output:
                display(df_head)  # Display the DataFrame inside the output widget
            tab_contents.append(table_output)  # Add to tab contents list

        # Create the tabs widget
        tabs = widgets.Tab(children=tab_contents)
        
        # Set tab titles
        for i, title in enumerate(tab_titles):
            tabs.set_title(i, title)

        # Display the tabs
        display(tabs)

This code cell displays multible tabs, each one displaying the first ten rows of a table

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
display_dataframes_in_tabs(___)  # Pass the keys of the dictionary to the function

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
You need to pass the keys of the dictionary `my_tables_dict` to the function `display_dataframes_in_tabs`. Use the `.keys()` method to get the keys.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
display_dataframes_in_tabs(my_tables_dict.keys())
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

# Introduction to Python Client teradataml & tdml DataFrames

## teradataml DataFrame Creation + Properties


This code cell imports the teradataml package.

In [ ]:
import teradataml as tdml

This code cell creates DataFrame objects from tables in the Teradata database, including creating from table name in the default database, from a query, and from a table in a non-default database. It also inspects the created DataFrame objects.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
#create from tablename in current default database
DF_Customer_Dim = tdml.DataFrame("___")  # Specify the table name

#create from query
DF_Account_Dim = tdml.DataFrame.from_query("SELECT * FROM ___")  # Complete the query with the table name

#create from tablename in non-default database, this is preferred
DF_Account_Customer_Map = tdml.DataFrame(tdml.in_schema("___", "___"))  # Specify the schema and table name

# inspect
DF_Customer_Dim

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
For each DataFrame creation, ensure you specify the correct table name or schema. The first DataFrame is created directly from a table name, the second from a SQL query, and the third from a table in a specified schema.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
#create from tablename in current default database
DF_Customer_Dim = tdml.DataFrame("Customer_Dim")

#create from query
DF_Account_Dim= tdml.DataFrame.from_query("SELECT * FROM Account_Dim")

#create from tablename in non-default database, this is preferred
DF_Account_Customer_Map = tdml.DataFrame(tdml.in_schema("demo_user", "Account_Customer_Map" ))

# inspect
DF_Customer_Dim
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

***DataFrame Properties***

This code cell retrieves the shape of the DataFrame DF_Customer_Dim.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
DF_Customer_Dim.___  # Access the shape attribute of the DataFrame

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To find the dimensions of a DataFrame, use the `.shape` attribute. 
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
DF_Customer_Dim.shape
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell retrieves the column names of the DataFrame DF_Customer_Dim.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
DF_Customer_Dim.___  # Access the columns attribute of the DataFrame

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To view the column names of a DataFrame, use the `columns` attribute. This attribute does not require parentheses as it is not a method.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
DF_Customer_Dim.columns
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell retrieves the data types of each column in the DataFrame DF_Customer_Dim.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
DF_Customer_Dim.___  # Check the data types of the DataFrame columns

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
Use the `dtypes` attribute to view the data types of each column in a DataFrame.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
DF_Customer_Dim.dtypes
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell attempts to retrieve the Teradata data types of the columns in the DataFrame DF_Customer_Dim.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
DF_Customer_Dim.___  # Use the appropriate method to check the Teradata data types of the DataFrame

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To check the Teradata data types of each column in a DataFrame, use the property (tdtypes) that provides this information. 
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
DF_Customer_Dim.tdtypes
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## SQL within Python (1): dynamic query building using format strings & `tdml.DataFrame.from_query()` 

This code cell constructs a SQL query using format strings and a list of conditions. The final query is printed and executed using the teradataml package.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
# use format strings to construct queries
query_f = "SELECT Customer_Id FROM Customer_Dim WHERE {conditions} "
conditions_list = ["Preferred_Language_Code = 'EN' ",
                          "Region IS IN ('BE', 'ZH') "]

query_final = query_f.format(conditions= " AND ".join(___))  # Join conditions with 'AND'

prettyprint_sql(___)  # Pretty print the final query

tdml.DataFrame.from_query(___)  # Create DataFrame from the query

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To construct the final query, you need to join the conditions in `conditions_list` with 'AND'. Then, use the `prettyprint_sql` function to display the query and `tdml.DataFrame.from_query` to execute it.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
# use format strings to construct queries
query_f = "SELECT Customer_Id FROM Customer_Dim WHERE {conditions} "
conditions_list = ["Preferred_Language_Code = 'EN' ",
                          "Region IS IN ('BE', 'ZH') "]

query_final = query_f.format(conditions= " AND ".join(conditions_list))

prettyprint_sql(query_final)

tdml.DataFrame.from_query(query_final)
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## Filtering DataFrames

This code filters the `DF_Customer_Dim` DataFrame based on the conditions that the Preferred Language Code is "EN" and the Region is either "BE" or "ZH". It then selects only the Customer_Id column and stores the result in `DF_myfilter`.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
DF_myfilter = DF_Customer_Dim[
    (DF_Customer_Dim["Preferred_Language_Code"] == ___) &  # Filter for English language, EN
    (DF_Customer_Dim["Region"].isin([___, ___]))  # Filter for specific regions, BE, ZH
][[___]]  # Select the Customer_Id column

DF_myfilter

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
You need to filter the DataFrame based on the preferred language being "EN" and the region being either 'BE' or 'ZH'. After filtering, select only the "Customer_Id" column.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
DF_myfilter = DF_Customer_Dim[
    (DF_Customer_Dim["Preferred_Language_Code"] == "EN") &
    (DF_Customer_Dim["Region"].isin(['BE', 'ZH']))
][["Customer_Id"]]

DF_myfilter
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## SQL within Python (2):  `show_query`, format strings,  `tdml.execute_sql()` to crystallise logic for later use

This code cell is used to inspect the SQL query behind the `DF_Account_Dim` DataFrame using the `prettyprint_sql` function.

In [ ]:
# inspect the query behind a DF, not interesting
prettyprint_sql(DF_Account_Dim.show_query())

This code cell is used to inspect the SQL query behind the processed `DF_myfilter` DataFrame using the `prettyprint_sql` function.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
# inspect the query behind a processed DF 
prettyprint_sql(DF_myfilter.____)  # Use the method to show the query of the DataFrame

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
You need to call the `show_query()` method on the DataFrame object to retrieve the SQL query. Make sure to pass the correct DataFrame variable to the `prettyprint_sql` function.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
# inspect the query behind a processed DF 
prettyprint_sql(DF_myfilter.show_query())
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell replaces a view named `customer_BEZH_EN` with the query of the `DF_myfilter` DataFrame. It then executes the SQL query to create the view and retrieves the DataFrame associated with the view.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
replace_view_f = """REPLACE VIEW {view_name} AS
{query}"""
view_name = "customer_BEZH_EN"  # Define the name of the view

replace_view_ddl = replace_view_f.format(view_name = view_name, 
                                         query = ___.___)  # Use the method to get the query
prettyprint_sql(replace_view_ddl)  # Pretty print the SQL statement

tdml.execute_sql(replace_view_ddl)  # Execute the SQL statement

tdml.DataFrame(___)  # Create a DataFrame from the view name

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
First, define the `view_name` variable with the appropriate view name. Then, ensure you call the `show_query()` method on the correct object to get the SQL query. Use the formatted SQL statement in the `prettyprint_sql`, `execute_sql`, and `DataFrame` functions.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
replace_view_f ="""REPLACE VIEW {view_name} AS
{query}"""
view_name = "customer_BEZH_EN"


replace_view_ddl = replace_view_f.format(view_name = view_name, 
                                         query = DF_myfilter.show_query())
prettyprint_sql(replace_view_ddl)

tdml.execute_sql(replace_view_ddl)

tdml.DataFrame(view_name)
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## Simple Joins

This code retrieves the DataFrames `DF_Transaction_Fact` and `DF_Account_Customer_Map` using the `tdml.DataFrame` function to join transactions with customer_dim via the account_customer map.

In [ ]:
# join transactions with customer_dim via account_customer map
DF_Transaction_Fact = tdml.DataFrame("Transaction_Fact")

DF_Account_Customer_Map = tdml.DataFrame("Account_Customer_Map")

This code cell references the DataFrame DF_Customer_Dim.

In [ ]:
DF_Customer_Dim

This code cell references the DataFrame DF_Account_Customer_Map.

In [ ]:
DF_Account_Customer_Map

This code cell joins the DF_Transaction_Fact with DF_Account_Customer_Map and DF_Customer_Dim. It drops unnecessary columns and displays the SQL query used for the join operation.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
DF_transaction_joined = DF_Transaction_Fact.join(
    DF_Account_Customer_Map, 
    on = ___,  # Specify the column to join on
    how = ___,  # Specify the type of join
    rprefix = "mymap"  # Specify the prefix for the right DataFrame
).join(
    DF_Customer_Dim,
    on = ___,  # Specify the column to join on
    how = ___,  # Specify the type of join
    lprefix = "mymap"  # Specify the prefix for the left DataFrame
).drop(columns=["mymap_Account_Id","mymap_Customer_Id"])  # Drop unnecessary columns

prettyprint_sql(DF_transaction_joined.show_query())

DF_transaction_joined

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
When joining DataFrames, ensure you specify the correct column names for the `on` parameter and choose the appropriate join type (`left`, `right`, `inner`, etc.). Use `rprefix` and `lprefix` to avoid column name conflicts. Finally, drop any columns that are not needed in the final DataFrame.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
DF_transaction_joined = DF_Transaction_Fact.join(
    DF_Account_Customer_Map, 
    on = "Account_Id", 
    how="left", rprefix ="mymap"
).join(
    DF_Customer_Dim,
    on = "Customer_Id",
    how = "left", lprefix ="mymap"
).drop(columns=["mymap_Account_Id","mymap_Customer_Id"])

prettyprint_sql(DF_transaction_joined.show_query())

DF_transaction_joined
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell demonstrates an alternative approach to joining multiple tables using SQL query directly. It shows that proficient SQL users may find it faster to write the join operation in SQL rather than chaining operations in Python.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
tdml.DataFrame.____("""
SELECT 
    mytransaction.*,
    cust_dim.*
FROM 
    Transaction_Fact AS mytransaction
LEFT JOIN
    Account_Customer_Map AS mymap
ON
    (mytransaction.___ = mymap.___) 
LEFT JOIN
    Customer_Dim AS cust_dim
ON 
    (mymap.___ = cust_dim.___)  

"""   
)

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
When performing a SQL JOIN, ensure that the fields you are joining on are correctly matched. Look for the common fields between the tables involved in the JOIN operations. use from_query to get a DataFrame based on a query.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
tdml.DataFrame.from_query("""
SELECT 
    mytransaction.*,
    cust_dim.*
FROM 
    Transaction_Fact AS mytransaction
LEFT JOIN
    Account_Customer_Map AS mymap
ON
    (mytransaction.Account_id = mymap.Account_id) 
LEFT JOIN
    Customer_Dim AS cust_dim
ON 
    (mymap.Customer_Id = cust_dim.Customer_Id) 

"""   
)
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## DataFrame manipulation: simple aggregates

This code cell calculates aggregate values based on the joined DataFrame DF_transaction_joined. It groups by "Customer_Id" and calculates the sum and mean of "Transaction_Amount". The SQL query used for this aggregation is displayed.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
DF_tj = DF_transaction_joined
DF_tr_agg = DF_tj.groupby("____").agg(
    [DF_tj['Transaction_Amount'].____().alias("Total_Transaction_Amount"),  # Alias for total transaction amount
     DF_tj['Transaction_Amount'].____().alias("Average_Transaction_Amount"),  # Alias for average transaction amount
    ])

prettyprint_sql(DF_tr_agg.____())  # Show the SQL query for the aggregation

DF_tr_agg

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
You need to provide meaningful aliases for the aggregated columns. Think about what each aggregation represents. Also, ensure you call the method to display the SQL query for the DataFrame.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
DF_tj = DF_transaction_joined
DF_tr_agg = DF_tj.groupby("Customer_Id").agg(
    [DF_tj['Transaction_Amount'].sum().alias("Total_Transaction_Amount"),
     DF_tj['Transaction_Amount'].mean().alias("Average_Transaction_Amount"),
    ])

prettyprint_sql(DF_tr_agg.show_query())

DF_tr_agg
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## Count Uniques and remove duplicates


This code cell creates a DataFrame object named DF_master from the "Master_Table" table and prints the column names and shape of the DataFrame.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
DF_master = tdml.DataFrame(___)  # Create a DataFrame from table name "Master_Table"
print(DF_master.___)  # Print the columns of the DataFrame
print(DF_master.___)  # Print the shape of the DataFrame

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To create a DataFrame, you need to specify the table name as a string. Use the `columns` attribute to get the column names and the `shape` attribute to get the dimensions of the DataFrame.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
DF_master = tdml.DataFrame("Master_Table")
print(DF_master.columns)
print(DF_master.shape)
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell selects specific columns ("Customer_Id", "City_Name", "Common_Transaction_Type") from the DF_master DataFrame and counts the number of distinct values in each column.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
columns_count_unique = ["Customer_Id", "City_Name", "Common_Transaction_Type"]

DF_master._____(columns_count_unique  # Select the columns to count unique values
                ).count(___=True)  # Count distinct values

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To count distinct values in a DataFrame, ensure you *select* the correct columns and use the `distinct` parameter correctly in the `count` method.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
columns_count_unique = ["Customer_Id","City_Name","Common_Transaction_Type"]

DF_master.select(columns_count_unique
                ).count(distinct=True)
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

In this code cell, unique combinations of columns ("City_Name", "Common_Transaction_Type", "Primary_Currency") are selected from the DF_master DataFrame. The duplicates are then dropped, and the resulting DataFrame is converted to a pandas DataFrame, sorted, and displayed.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
columns_unique_combos = ["City_Name",
                         "Common_Transaction_Type",
                         "Primary_Currency"]
DF_combo = DF_master.select(___).drop_duplicate()  # Select and drop duplicates

# Aggregate to pandas 
df_combo = DF_combo.to_pandas(all_rows = ___)  # Convert to pandas DataFrame, make sure all rows are transmited
df_combo.sort_values(list(df_combo.columns))  # Sort values by all columns

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
1. Use the list `columns_unique_combos` to select specific columns from `DF_master`.
2. When converting to a pandas DataFrame, ensure all rows are included.
3. To sort the DataFrame, use the list of its columns.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
columns_unique_combos = ["City_Name",
                         "Common_Transaction_Type",
                         "Primary_Currency"]
DF_combo = DF_master.select(columns_unique_combos).drop_duplicate()

#aggreagte to pandas (only couple rows)
df_combo = DF_combo.to_pandas(all_rows = True)
df_combo.sort_values(list(df_combo.columns))
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## Select columns based on datatypes

This code cell retrieves the data types of columns in the DF_master DataFrame and categorizes them into integer, float, and string features. It then selects columns with float data types from the DF_master DataFrame.

In [ ]:
master_types = DF_master.dtypes._column_names_and_types
print(master_types)

int_feats = [c[0] for c in master_types if c[1] == "int"]
float_feats = [c[0] for c in master_types if c[1] == "float"]
str_feats = [c[0] for c in master_types if c[1] == "str"]

DF_master.select(float_feats)

## Create columns, constant values and case whens
https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-20.00/teradataml-Data-Preparation-Functions/Date-and-Time-Functions/current_date

This code cell adds constants (campaign_id, campaign_author, campaign_planning_date) to the DF_master DataFrame for a campaign. The SQL query for displaying the DataFrame is printed, and the modified DataFrame DF_campaign is displayed.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
# add constants to DF for a campaign
from sqlalchemy import func as f

thiscampaign_id = "f3354_002"
thiscampaign_author = "martin"

DF_campaign = DF_master.assign(
    campaign_id = ___,
    campaign_author = ___,
    campaign_planning_date = f.___()  # Use an SQL function to get the current date
)

prettyprint_sql(DF_campaign.___())  # Show the SQL query

DF_campaign

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
You need to assign values to `thiscampaign_id` and `thiscampaign_author`. Use `f.current_date()` to get today's date. To display the SQL query, use the `show_query()` method on `DF_campaign`.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
# add constants to DF for a campaign
from sqlalchemy import func as f

thiscampaign_id = "f3354_002"
thiscampaign_author = "martin"

DF_campaign = DF_master.assign(
    campaign_id = thiscampaign_id,
    campaign_author = thiscampaign_author,
    campaign_planning_date = f.current_date() # today
)

prettyprint_sql(DF_campaign.show_query())

DF_campaign
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell demonstrates how to manually set new columns in a DataFrame using different options such as full SQL, SQL literal_column, and DataFrame Column Expression. It assigns new columns based on certain conditions like Customer_Segment and Region. The resulting DataFrame is displayed and returned.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
from sqlalchemy.sql import literal_column as col
# works for both picking out a column and injecting in some SQL

# manual settings of new columns
DF_c2 = DF_campaign.___(
    # Option 1: FULL SQL
      flag_PB =  
          col("CASE WHEN Customer_Segment = 'Private Banking' THEN 1 ELSE 0 END"),
        
        # Option 2: pick out column with SQL literal_column
     flag_ZH =
        tdml.case([(___("Region") == ___ , 1)], else_=0),      
    
    # Option 3: pick out column with DataFrame Column Expression
    flag_BE =
        tdml.case([(DF_campaign.___ == 'BE' , 1)], else_=0),
)

prettyprint_sql(DF_c2.show_query())

DF_c2

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
For the `flag_ZH` and `flag_BE` columns, you need to specify the region code that should be checked against the "Region" column. The first one should check for "ZH" and the second one for "BE".
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
from sqlalchemy.sql import literal_column as col
# works for both picking out a column and injecting in some SQL

# manual settings of new columns
DF_c2 = DF_campaign.assign(
    # Option 1: FULL SQL
      flag_PB =  
          col("CASE WHEN Customer_Segment = 'Private Banking' THEN 1 ELSE 0 END"),
        
        # Option 2: pick out column with SQL literal_column
     flag_ZH =
        tdml.case([(col("Region") == "ZH" , 1)], else_=0),      
    
    # Option 3: pick out column with DataFrame Column Expression
    flag_BE =
        tdml.case([(DF_campaign.Region == "BE" , 1)], else_=0),
)

prettyprint_sql(DF_c2.show_query())

DF_c2
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell shows how to create a dictionary of indicator columns based on unique regions in a DataFrame. It uses a dictionary comprehension to generate columns dynamically for each region. The indicator columns are then assigned to the campaign DataFrame. The resulting DataFrame is displayed and returned.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
# get unique regions
my_regions = DF_master.select(
        [___]  # Select the "Region" column
    ).___().to_pandas()
my_regions

# create dict of same process
indicator_columns_dict = {

     "flag_"+region: tdml.case([(col("Region") == ___ , 1)], else_=0)  # Compare each region

    for region in ___.Region  # Iterate over the regions
}

DF_c3 = DF_campaign.___(**indicator_columns_dict)

prettyprint_sql(DF_c3.show_query())
DF_c3

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
First, ensure you are selecting the correct column name for regions. Then, when creating the dictionary, make sure to compare the correct column with each region. Finally, iterate over the correct attribute of the DataFrame to get the list of regions.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
# get unique regions
my_regions = DF_master.select(
        ["Region"]
    ).drop_duplicate().to_pandas()
my_regions

# create dict of same process
indicator_columns_dict = {

     "flag_"+region: tdml.case([(col("Region") == region , 1)], else_=0)      

    for region in my_regions.Region
}

DF_c3 = DF_campaign.assign(**indicator_columns_dict)

prettyprint_sql(DF_c3.show_query())
DF_c3
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## Random Sampling of Rows

In [ ]:
DF_Customer_Dim

In this code cell, a sample of 10,000 rows is taken from the DataFrame DF_Customer_Dim using the sample method. The sampling is randomized across AMPs in Teradata. The resulting sampled DataFrame is displayed and returned.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
DF_c_sample = DF_Customer_Dim.___(
    n= ___,  # Specify the number of samples to draw
    randomize = ___,  # Determine if sampling should be randomized across AMPs
                      )
prettyprint_sql(DF_c_sample.show_query())

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
Consider how many samples you want to draw from the DataFrame and whether you want the sampling to be randomized across AMPs. The `n` parameter specifies the number of samples, and `randomize` is a boolean indicating if the sampling should be randomized.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
DF_c_sample = DF_Customer_Dim.sample(
    n= 10000,
    randomize = True, #Specifies if sampling should be done across AMPs in Teradata or per AMP.
                      )
prettyprint_sql(DF_c_sample.show_query())
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell demonstrates consistent sampling with a hashing trick. It creates new columns 'myhashid' and 'myfilter' based on the hashing of Customer_Id with a salt. The 'myfilter' column is assigned based on a condition using the 'myhashid' column. The resulting DataFrame with the new columns is displayed and returned.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
# consistent sampling with hashing trick
# if you want to understand this "hashing trick", look at the blog post
# https://medium.com/@martinhillebrandtd/hash-your-way-to-data-science-glory-5cb07cea4d5f

DF_CusDim_Hash = DF_Customer_Dim.assign(
    myhashid = f.mod(f.abs(f.from_bytes(f.hashrow(DF_Customer_Dim.Customer_Id.expression + "salt"), "base10")), ___ ), # get the last digit of the integer
    myfilter = col("CASE WHEN ___ < ___ THEN 1 ELSE 0 END") # select 30 % of all rows
)

prettyprint_sql(DF_CusDim_Hash.show_query())

DF_CusDim_Hash

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To implement the hashing trick, you need to add a "salt" to the customer ID expression, determine the modulus value for the hash, and set a threshold for filtering. The modulus value determines the range of hash IDs, and the threshold decides which IDs are selected.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
# consistent sampling with hashing trick
# if you want to understand this "hashing trick", look at the blog post
# https://medium.com/@martinhillebrandtd/hash-your-way-to-data-science-glory-5cb07cea4d5f

DF_CusDim_Hash = DF_Customer_Dim.assign(
    myhashid = f.mod(f.abs(f.from_bytes(f.hashrow(DF_Customer_Dim.Customer_Id.expression + "salt"), "base10")),10 ),
    myfilter = col("CASE WHEN myhashid < 3 THEN 1 ELSE 0 END")
)

prettyprint_sql(DF_CusDim_Hash.show_query())

DF_CusDim_Hash
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

# Data Quality Analysis

## Count Missing Values

This code cell counts missing values in the "Customer_Details" DataFrame and sorts the result in descending order based on the number of missing values.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
# count missing values
DF_Customer_Details = tdml.DataFrame("Customer_Details")
ColumnSummary_Customer_Details = tdml.ColumnSummary(data = DF_Customer_Details, target_columns = ___.___, # all columns should be used
                  volatile = True
                  )

ColumnSummary_Customer_Details.result.sort("NullCount", ascending = ___) # sort so that columns with most NULLs appear on top

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
1. Use the `columns` attribute of the DataFrame to specify the target columns.
3. The `sort` method is used to order the results by the count of null values; ensure you specify the correct order.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
# count missing values
DF_Customer_Details =  tdml.DataFrame("Customer_Details")
ColumnSummary_Customer_Details  = tdml.ColumnSummary(data = DF_Customer_Details, target_columns = DF_Customer_Details.columns,
                  volatile = True
                  )

ColumnSummary_Customer_Details.result.sort("NullCount", ascending = False)
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## Remove missing values

This code cell deletes rows with missing values in the "Street_Name" and "House_Number" columns of the "Customer_Details" DataFrame, prints the shape of the resulting DataFrame, and shows the SQL query for the operation.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
# Delete missing values
DF_noNulls = DF_Customer_Details.dropna(subset=[___, ___])  # Specify columns to check for missing values
print(DF_noNulls.___)  # Print the shape of the DataFrame
prettyprint_sql(DF_noNulls.show_query())

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To remove rows with missing values, specify the column names in the `subset` parameter of `dropna`. Use `shape` to get the dimensions of the DataFrame and `show_query` to display the SQL query.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
# Delete missing values
DF_noNulls = DF_Customer_Details.dropna(subset=["Street_Name","House_Number"] )
print(DF_noNulls.shape)
prettyprint_sql(DF_noNulls.show_query())
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## indicator columns for missing values

This code cell creates indicator columns for missing values in the "Customer_Details" DataFrame based on each column, assigns these indicator columns to a new DataFrame, and displays the SQL query for the operation.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
# indicator columns for missing values
this_cols = DF_Customer_Details.columns[1:]

def get_missing_indicator(colname):
    return f.decode(col(___).expression,  # Use the column name
                 None, ___,  # if null then 1
                 ___)  # else 0
    
missingindiciator_columns_dict = {
    "missing_"+this_col: get_missing_indicator(this_col) 
    for this_col in ___  # Iterate over the columns
}

print(missingindiciator_columns_dict)

DF_missingindicator_Customer_Details = DF_Customer_Details.assign(
    drop_columns = True,
    Customer_Id = col("Customer_Id"),
    ___ ____ # uses the kwargs trick
)
quality_query = DF_missingindicator_Customer_Details.show_query()
prettyprint_sql(quality_query)
DF_missingindicator_Customer_Details

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To create a missing indicator, use the `decode` function to check if a column value is null. If it is null, return 1; otherwise, return 0. Make sure to iterate over the correct columns and use the column name in the function. To use a dictionary as a named parameterlist you can use the double asterix **
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
# indicator columns for missing values
this_cols = DF_Customer_Details.columns[1:]

def get_missing_indicator(colname):
    return f.decode(col(colname).expression, 
                 None, 1, # if null then 1
                 0) # else 0
    
missingindiciator_columns_dict = {
    "missing_"+this_col:get_missing_indicator(this_col) 
    for this_col in this_cols
}


print(missingindiciator_columns_dict)

DF_missingindicator_Customer_Details = DF_Customer_Details.assign(
    drop_columns = True,
    Customer_Id = col("Customer_Id"),
    **missingindiciator_columns_dict
)
quality_query = DF_missingindicator_Customer_Details.show_query()
prettyprint_sql(quality_query)
DF_missingindicator_Customer_Details
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell replaces a view named "quality_customerdetails" with the SQL query generated in the previous code cell, and then sorts the "quality_customerdetails" DataFrame in descending order based on the "missing_House_Number" column.

In [ ]:
tdml.execute_sql(f"REPLACE VIEW quality_customerdetails AS ({quality_query})")

tdml.DataFrame("quality_customerdetails").sort("missing_House_Number", ascending = False)

# Descriptive Statistics

## Frequencies of categorial variables

This code cell identifies categorical variables in the "DF_master" DataFrame, removes the "Customer_Id" column from the list of categorical variables, generates a categorical summary for these variables, and sorts the result based on the column name and distinct value count.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
categorical_variables =  [c[0] for c in DF_master.dtypes._column_names_and_types if c[1] == ___]  # Check for string type

categorical_variables.remove("Customer_Id")

CategoricalSummary = tdml.CategoricalSummary(data = DF_master, target_columns= ___, volatile = True)  # Use the list of categorical variables

DF_CatSum_Master = CategoricalSummary.result
DF_CatSum_Master.sort(["ColumnName", "DistinctValueCount"])  # Sort by column name and distinct value count

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">

1. To identify categorical variables, check for columns with data type "str".

2. Use the list of categorical variables as the target columns in the `CategoricalSummary`.

  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
categorical_variables =  [c[0] for c in DF_master.dtypes._column_names_and_types if c[1] == "str"]

categorical_variables.remove("Customer_Id")

CategoricalSummary = tdml.CategoricalSummary(data = DF_master, target_columns= categorical_variables, volatile = True)

DF_CatSum_Master = CategoricalSummary.result
DF_CatSum_Master.sort(["ColumnName","DistinctValueCount"])
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell adds columns to a DataFrame based on the original report. It calculates cumulative count, sum of counts, percentage, and cumulative percentage for each column. The DataFrame is then sorted by "ColumnName" and "DistinctValue".

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
# add columns as per original report

df_catsum_master = DF_CatSum_Master.assign(
    cum_count = DF_CatSum_Master["DistinctValueCount"].window(
                    partition_columns="___",  # Specify the column to partition by
                    order_columns = "___",  # Specify the column to order by
                    window_end_point = 0  
                    ).sum(),
    sum_counts = DF_CatSum_Master["DistinctValueCount"].window(partition_columns="___").sum(),  # Specify the partition column
    percentage = f.round((col("DistinctValueCount") / col("___") ) *100,2),  # Use the correct column 
    cum_percentage = f.round((col("cum_count") / col("___") ) *100,2),  # Use the correct column 
).to_pandas()

df_catsum_master = df_catsum_master.sort_values(["ColumnName","DistinctValue"])

df_catsum_master.head()

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
When using window functions, ensure you specify the correct columns for partitioning and ordering. The `window_end_point` is typically set to 0 for cumulative calculations. For sorting, use the columns that define the order of your data.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
# add columns as per original report

df_catsum_master = DF_CatSum_Master.assign(
    cum_count = DF_CatSum_Master["DistinctValueCount"].window(
                    partition_columns="ColumnName",
                    order_columns = "DistinctValue",
                    window_end_point = 0
                    ).sum(),
    sum_counts = DF_CatSum_Master["DistinctValueCount"].window(partition_columns="ColumnName").sum(),
    percentage = f.round((col("DistinctValueCount") / col("sum_counts") ) *100,2),
    cum_percentage = f.round((col("cum_count") / col("sum_counts") ) *100,2),
).to_pandas()

df_catsum_master = df_catsum_master.sort_values(["ColumnName","DistinctValue"])

df_catsum_master.head()
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell defines a function to generate a categorical summary for a DataFrame. It identifies categorical columns, computes additional metrics, creates a plot showing the frequency of categories for each column, and returns the summary DataFrame and the plot. The function is then called with a DataFrame and specific columns to ignore.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
from sqlalchemy.sql import literal_column 
# add constants to DF for a campaign
from sqlalchemy import func as f

def generate_categorical_summary(this_DF, ignore_columns = ["Customer_id"]):
    # Identify categorical columns
    categorical_variables = [c[0] for c in this_DF.dtypes._column_names_and_types if c[1] == "str"]
    categorical_variables = [c for c in categorical_variables if c not in ignore_columns]
    
    # Generate Categorical Summary
    CategoricalSummary = tdml.CategoricalSummary(data=this_DF, target_columns=categorical_variables)
    DF_CatSum = CategoricalSummary.result

    # Compute additional metrics
    
    DF_CatSum2 = DF_CatSum.assign(
        cum_count=DF_CatSum["DistinctValueCount"].window(
            partition_columns="ColumnName",
            order_columns="DistinctValue",
            window_end_point=0
        ).sum(),
        sum_counts=DF_CatSum["DistinctValueCount"].window(partition_columns="ColumnName").sum(),
        percentage=f.round((literal_column("DistinctValueCount") / literal_column("sum_counts")) * 100, 2),
        cum_percentage=f.round((literal_column("cum_count") / literal_column("sum_counts")) * 100, 2)
    )
    
    df_catsum = DF_CatSum2.to_pandas()
    df_catsum = df_catsum.sort_values(["ColumnName", "DistinctValue"])

    # Create plot
    fig = make_subplots(rows=len(categorical_variables), cols=1, shared_xaxes=False, subplot_titles=categorical_variables)
    for i, col in enumerate(categorical_variables, start=1):
        df_col = df_catsum[df_catsum['ColumnName'] == col]
        fig.add_trace(
            go.Bar(
                x=df_col['DistinctValue'],
                y=df_col['DistinctValueCount'],
                name=col,
                marker_color=px.colors.qualitative.Plotly[:len(df_col['DistinctValue'])],
                showlegend=False
            ),
            row=i, col=1
        )
    
    fig.update_layout(height=300*len(categorical_variables), title="Frequency of Categories for Each Column")
    #fig.show()
    
    return df_catsum, fig
      

df_catsum, fig = generate_categorical_summary(DF_Customer_Dim, ignore_columns=["Account_Id", "Customer_Id"])

print(df_catsum.head())

fig.show()

## Binning of continuous variables

https://docs.teradata.com/r/Enterprise_IntelliFlex_Lake_VMware/Vantage-Analytics-Library-User-Guide/Descriptive-Statistics

https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-20.00/teradataml-Vantage-Analytics-Library-Functions



This code cell creates a DataFrame object for the "Transaction_Fact" table in the "demo_user" schema.

In [ ]:
DF_Transaction_Fact = tdml.DataFrame(tdml.in_schema("demo_user","Transaction_Fact"))

DF_Transaction_Fact

### equal width binning with Histogramm

This code cell creates a histogram using equal-width binning for the "Transaction_Amount" column in the "DF_Transaction_Fact" DataFrame. It calculates the cumulative percentage for each bin and sorts the DataFrame by label.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
Hist_eqw = tdml.Histogram(
    data = ___,  # Provide the DataFrame containing the data
    target_columns= ["Transaction_Amount"],
    method_type= "EQUAL-WIDTH", nbins = ___,  # Specify the number of bins, should be 10.
    volatile = True)
           
DF_hist = Hist_eqw.result

DF_hist2 = DF_hist.assign(Bin_Percent_cumulative = DF_hist.Bin_Percent.___(["Label"])).sort("Label") # choose the function that can do cumulative sums

DF_hist2

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
Ensure you specify the DataFrame that contains the transaction data and decide on the number of bins for the histogram. When sorting, consider which column you want to use to order the data.
      
`csum` is what you are looking for.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
Hist_eqw = tdml.Histogram(
    data = DF_Transaction_Fact,
    target_columns= ["Transaction_Amount"],
    method_type= "EQUAL-WIDTH", nbins = 10, volatile = True)
           

DF_hist = Hist_eqw.result

DF_hist2 = DF_hist.assign(Bin_Percent_cumulative = DF_hist.Bin_Percent.csum(["Label"])).sort("Label")

DF_hist2
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

### Variable Width binning with Histogramm, predefined

https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-20.00/teradataml-Vantage-Analytics-Library-Functions/Descriptive-Statistics/Histogram

This code cell uses the histogram function from Teradata's Vantage Analytic Library (VAL) to create a histogram for the "Transaction_Amount" column in the "DF_Transaction_Fact" DataFrame. It calculates the cumulative percentage for each bin, materializes the DataFrame, and sorts it by bin.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
# we use the histogram function from teradata's Vantage Analytic Library (VAL)
tdml.configure.val_install_location = "val"  # Set the VAL installation location

histobj = tdml.valib.Histogram(data= DF_Transaction_Fact, columns =["Transaction_Amount"], boundaries=[1,500,2000,5000])  # Create a histogram object

DF_hist = histobj.___  # Get the result of the histogram

DF_hist = DF_hist.assign(Bin_Percent_cumulative = DF_hist.xpct.___(["xbin"]))  # Calculate cumulative percentage

DF_hist = DF_hist.___()  # Materialize the DataFrame

DF_hist.sort("xbin")  # Sort the DataFrame by 'xbin'

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
1. Set the VAL installation location using a string.
2. Use the appropriate DataFrame as input data for the histogram.
3. Access the result attribute of the histogram object.
4. Use the cumulative sum function to calculate the cumulative percentage.
5. Materialize the DataFrame to finalize the changes.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
# we use the histogram function from teradata's Vantage Analytic Library (VAL)
tdml.configure.val_install_location = "val"

histobj = tdml.valib.Histogram(data= DF_Transaction_Fact,columns =["Transaction_Amount"], boundaries=[1,500,2000,5000])

DF_hist = histobj.result

DF_hist = DF_hist.assign(Bin_Percent_cumulative = DF_hist.xpct.csum(["xbin"]))

DF_hist = DF_hist.materialize()

DF_hist.sort("xbin")
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

### Variable Width Quantile based

This code cell creates a histogram object using the Teradata Python package. It specifies the data source, columns to be used for the histogram, and the number of quantiles to be generated.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
histobj = tdml.valib.Histogram(data= DF_Transaction_Fact, columns =["___"], quantiles=___) # we want to have 8 quantiles

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
You need to specify the data source, the column you want to create a histogram for, and the number of quantiles. The data source is likely a DataFrame, and the column name should match one in the DataFrame.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
histobj = tdml.valib.Histogram(data= DF_Transaction_Fact,columns =["Transaction_Amount"], quantiles=8)
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell retrieves the result of the histogram object created in the previous cell, converts it to a Pandas DataFrame, sorts the data based on the 'xbeg' column, extracts the quantile values, and prints them. It also sorts the DataFrame based on the 'xbeg' column.

In [ ]:
DF_hist_quantile = histobj.result
df_hist_quantile = DF_hist_quantile.to_pandas().reset_index()
df_hist_quantile = df_hist_quantile.sort_values("xbeg")
quantiles = list(df_hist_quantile.xbeg)[1:]
print("thresholds:", str(quantiles))
DF_hist_quantile.sort("xbeg")

### bin a column based on thresholds

This code cell defines a function that bins the values of a specified column in a DataFrame based on given thresholds. It creates bins for values less than or equal to the first threshold, values falling within each threshold range, values greater than the last threshold, and NULL values.

In [ ]:
def get_column_binned(thisDF, column_name, thresholds):
    thiscol = thisDF[column_name]
    bins = []
    
    # Add the first bin (values <= first threshold)
    bins.append((thiscol <= thresholds[0], f"bin_{str(0).zfill(3)}: ]-∞;{thresholds[0]}]"))
    
    # Add the middle bins
    for i in range(len(thresholds) - 1):
        lower_th, upper_th = thresholds[i], thresholds[i + 1]
        bin_number = str(i + 1).zfill(3)
        bins.append(((thiscol > lower_th) & (thiscol <= upper_th), f"bin_{bin_number}: ]{lower_th};{upper_th}]"))
    
    # Add the last bin (values > last threshold)
    bin_number = str(len(thresholds)).zfill(3)
    bins.append((thiscol > thresholds[-1], f"bin_{bin_number}: ]{thresholds[-1]};+∞]"))

    # Add the NULL bin 
    bin_number = str(len(thresholds)+1).zfill(3)
    bins.append((thiscol == None, f"bin_{bin_number}: NULL"))
    
    return tdml.case(bins)


This code cell assigns a new column to the DataFrame 'DF_Transaction_Fact' by binning the 'Transaction_Amount' column using the function defined in a previous cell. It then displays the SQL query generated for the DataFrame and shows the resulting DataFrame with the binned column.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
DF_binned = DF_Transaction_Fact.____( # add new columns
    Transaction_Amount_binned = get_column_binned(DF_Transaction_Fact, "Transaction_Amount", thresholds=___))  # Specify the thresholds for binning
prettyprint_sql(DF_binned.show_query())  # Show the SQL query for the binned DataFrame
DF_binned

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To bin the "Transaction_Amount" column, you need to specify the thresholds, which are typically quantiles or specific values. After binning, use the `show_query()` method to display the SQL query representation of the DataFrame.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
DF_binned = DF_Transaction_Fact.assign(
    Transaction_Amount_binned = get_column_binned(DF_Transaction_Fact,"Transaction_Amount", thresholds=quantiles))
prettyprint_sql(DF_binned.show_query())
DF_binned
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell generates a categorical summary of the 'Transaction_Amount_binned' column from the DataFrame 'DF_binned' and displays the summary along with a figure.

In [ ]:
df_catsum, fig = generate_categorical_summary(DF_binned.select(["Transaction_Amount_binned"]))

print(df_catsum.head())

fig.show()

# Waterfall Analysis: Aggregation & Visualisation

## Define Data and Filters

This code cell creates a Teradata DataFrame object named 'DF_master' based on the table 'Master_Table'.

In [ ]:
DF_master = tdml.DataFrame("Master_Table")

In this code cell, a list of conditions (CND) is defined to filter the data based on various criteria. The filters include region, segment, class, language, accounts, currency, transaction amount, balance, channel, interaction type, interactions, lowest balance, transaction type, city, and average transaction amount. The filter columns are extracted from the conditions.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
CND = [
    ("region_filter", "Region NOT IN ('GE', 'VD')"),
    ("segment_filter", "Customer_Segment NOT IN ('Private Banking')"),
    ("class_filter", "Customer_Class <> 'A'"),
    ("language_filter", "Preferred_Language_Code IN ('EN', 'DE')"),
    ("accounts_filter", "Num_Accounts > 1"),
    ("currency_filter", "Primary_Currency NOT IN ('EUR', 'USD')"),
    ("transaction_amount_filter", "Total_Transaction_Amount > 50000"),
    ("balance_filter", "Current_Balance IS NOT NULL"),
    ("channel_filter", "Preferred_Channel <> 'Phone'"),
    ("interaction_type_filter", "Frequent_Interaction_Type IN ('Inquiry', 'Service Request')"),
    ("interactions_filter", "Total_Interactions >= 5"),
    ("lowest_balance_filter", "Lowest_Balance > 0"),
    ("transaction_type_filter", "Common_Transaction_Type = 'Credit'"),
    ("city_filter", "City_Name NOT IN ('Zurich', 'Geneva')"),
    ("avg_transaction_filter", "Average_Transaction_Amount < 10000")
]
filter_cols = [c[_] for c in ___]  # Extract the first element of each tuple in the list, to have a list of column names

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To extract the first element from each tuple in a list, you can use a list comprehension. The list `CND` contains tuples, and you need to iterate over them to get the first item of each tuple. We do not start counting at 1....
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
CND = [
    ("region_filter", "Region NOT IN ('GE', 'VD')"),
    ("segment_filter", "Customer_Segment NOT IN ('Private Banking')"),
    ("class_filter", "Customer_Class <> 'A'"),
    ("language_filter", "Preferred_Language_Code IN ('EN', 'DE')"),
    ("accounts_filter", "Num_Accounts > 1"),
    ("currency_filter", "Primary_Currency NOT IN ('EUR', 'USD')"),
    ("transaction_amount_filter", "Total_Transaction_Amount > 50000"),
    ("balance_filter", "Current_Balance IS NOT NULL"),
    ("channel_filter", "Preferred_Channel <> 'Phone'"),
    ("interaction_type_filter", "Frequent_Interaction_Type IN ('Inquiry', 'Service Request')"),
    ("interactions_filter", "Total_Interactions >= 5"),
    ("lowest_balance_filter", "Lowest_Balance > 0"),
    ("transaction_type_filter", "Common_Transaction_Type = 'Credit'"),
    ("city_filter", "City_Name NOT IN ('Zurich', 'Geneva')"),
    ("avg_transaction_filter", "Average_Transaction_Amount < 10000")
]
filter_cols = [c[0] for c in CND]
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell applies the defined filters from CND to the 'DF_master' DataFrame and assigns binary values (1 or 0) based on whether the conditions are met. The resulting DataFrame is then converted to a temporary table 'master_filter' in Teradata.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
from sqlalchemy.sql import literal_column as col

DF_master_filter = DF_master.assign(
    **{filter_name: col(f"___ ___ ({filter_sql}) ___ 1 ___ 0 END")  # which SQL construct could be used here?
       for (filter_name, filter_sql) in ___}  # Provide the correct iterable for filter names and SQL
)
prettyprint_sql(DF_master_filter.show_query())

DF_master_filter.to_sql("master_filter", temporary=True, types={c: tdml.BYTEINT for c in filter_cols})  # Specify the columns for types
DF_master_filter = tdml.DataFrame("master_filter")  # Provide the correct table name
DF_master_filter

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
You need to fill in the SQL condition for filtering, the iterable that provides filter names and SQL conditions, the columns for which types are specified, and the table name for creating a DataFrame. Look at how SQL conditions are structured and how DataFrames are created and manipulated. You might want to remember CASE WHEN THEN ELSE END.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
from sqlalchemy.sql import literal_column  as col

DF_master_filter = DF_master.assign(
    **{filter_name: col(f"CASE WHEN ({filter_sql}) THEN 1 ELSE 0 END") 
       for (filter_name, filter_sql) in CND}
)
prettyprint_sql(DF_master_filter.show_query())

DF_master_filter.to_sql("master_filter", temporary = True,types= {c:tdml.BYTEINT for c in filter_cols})
DF_master_filter = tdml.DataFrame("master_filter")
DF_master_filter
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## Create Pre-Count Tables of all Combinations

In this code cell, a new DataFrame 'DF_WF' is created by grouping the 'DF_master_filter' DataFrame based on the filter columns and counting the number of unique 'Customer_Id' values for each combination of filters. The resulting DataFrame is then saved as a table 'waterfall_base' in Teradata.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
num_rows = 2**len(___)  # Calculate the number of rows based on filter columns
print(f"DF_WF will have maximum {num_rows} rows, as per the number of combinations of all filters is 2**({len(___)})")
DF_WF = (DF_master_filter
     .select(["Customer_Id"]+___)  # Select Customer_Id and filter columns
     .groupby(___)  # Group by filter columns
     .agg(DF_master_filter.Customer_Id.___().alias("number_customers"))  # Aggregate to count customers
)
prettyprint_sql(DF_WF.show_query())  # Print the SQL query

DF_WF.to_sql("waterfall_base", primary_index=___, if_exists= "replace")  # Save to SQL with primary index

DF_WF = tdml.DataFrame("waterfall_base")  # Load the DataFrame from SQL

DF_WF  # Display the DataFrame

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
Identify the variable that holds the filter columns and use it consistently in the code. Remember to select, group by, and set the primary index using these filter columns.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
num_rows = 2**len(filter_cols)
print(f"DF_WF will have maximum {num_rows} rows, as per the number of combinations of all filters is 2**({len(filter_cols)})")
DF_WF = (DF_master_filter
     .select(["Customer_Id"]+filter_cols)
     .groupby(filter_cols)
     .agg(DF_master_filter.Customer_Id.count().alias("number_customers"))
)
prettyprint_sql(DF_WF.show_query())

DF_WF.to_sql("waterfall_base", primary_index=filter_cols, if_exists= "replace")

DF_WF = tdml.DataFrame("waterfall_base")

DF_WF
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell retrieves the sum of the number of customers from a table called waterfall_base based on all filter columns. The filter columns are specified in the filter_cols list. The objective is to get the total number of customers based on the provided filters.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
# Aggregate function to sum up the customers
# Right arithmetic operation for the where condition
tdml.DataFrame.from_query(f"""
    SELECT 
        ___(number_customers) as number_customers   
    FROM
        waterfall_base 
    WHERE {" ___ ".join(filter_cols)} = {len(filter_cols)}
""")

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
To complete the SQL query, remember to use the correct column name for summing up the customers and specify the table from which you are selecting. The WHERE clause should ensure that the sum of the filter columns equals the number of columns.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python

tdml.DataFrame.from_query(f"""
    SELECT 
        SUM(number_customers) as number_customers
    FROM
        waterfall_base
    WHERE {" + ".join(filter_cols)} = {len(filter_cols)}
""")
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## Calculate number of remaining rows after applying sequence of filters

This code cell defines three functions: get_number_customers_filter, get_number_customers_total, and get_waterfall_sequence. These functions are used to calculate the total number of customers, the number of customers after applying filters, and the sequence of customer reduction based on filters. The get_waterfall_sequence function utilizes the other two functions to generate the sequence.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
def get_number_customers_filter(filter_columns, tablename = "waterfall_base"):
    return tdml.execute_sql(f"""
        SELECT 
            ZEROIFNULL(SUM(number_customers)) as number_customers
        FROM
            {tablename}
        WHERE {" + ".join(filter_columns)} = {len(filter_columns)}
    """).fetchall()[0][0]

def get_number_customers_total(tablename = "waterfall_base"):
    return tdml.execute_sql(f"""
        SELECT 
            ZEROIFNULL(SUM(number_customers)) as number_customers
        FROM
            {tablename}
    """).fetchall()[0][0]

def get_waterfall_sequence(filter_columns, tablename = "waterfall_base"):
    total_customers = get_number_customers_total(tablename)
    customers_after_filters = []
    for i in range(1,len(filter_columns)+1):
        customers_after_filters.append(
            get_number_customers_filter(filter_columns[:i], tablename))

    return total_customers, customers_after_filters
    

get_waterfall_sequence(___, tablename = "waterfall_base")  # Provide the filter columns

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
 In the `get_waterfall_sequence` function call, provide the list of filter columns to be used.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
def get_number_customers_filter(filter_columns, tablename = "waterfall_base"):
    return tdml.execute_sql(f"""
        SELECT 
            ZEROIFNULL(SUM(number_customers)) as number_customers
        FROM
            {tablename}
        WHERE {" + ".join(filter_columns)} = {len(filter_columns)}
    """).fetchall()[0][0]

def get_number_customers_total(tablename = "waterfall_base"):
    return tdml.execute_sql(f"""
        SELECT 
            ZEROIFNULL(SUM(number_customers)) as number_customers
        FROM
            {tablename}
    """).fetchall()[0][0]

def get_waterfall_sequence(filter_columns, tablename = "waterfall_base"):
    total_customers = get_number_customers_total(tablename)
    customers_after_filters = []
    for i in range(1,len(filter_columns)+1):
        customers_after_filters.append(
            get_number_customers_filter(filter_columns[:i], tablename))

    return total_customers, customers_after_filters
    

get_waterfall_sequence(filter_cols, tablename = "waterfall_base")
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## create plotly waterfall chart

This code cell creates a waterfall chart using the Plotly library. The chart visualizes the reduction in the number of customers based on different filters. The create_waterfall_chart function prepares the data for the chart and generates the visualization. It takes the Conditions and Decisions (CND), total customers, and customers after filters as input.

In [ ]:
import plotly.graph_objects as go

def create_waterfall_chart(CND, total_customers, customers_after_filters ):
    
    # Prepare data for the waterfall chart
    x_labels = [f"{name}\n{condition}" for name, condition in CND]  # Combine name and SQL condition
    y_values = [total_customers] + customers_after_filters  # Start with the total, then apply each filter
    x_labels.append("Remaining Customers")  # Add label for the final bar
    remaining_customers = customers_after_filters[-1]
    
    # Calculate the differences for the waterfall bars
    diff_values = [y_values[0] - y_values[1]]  # Initial drop from total to the first filter
    diff_values += [y_values[i] - y_values[i + 1] for i in range(1, len(y_values) - 1)]
    diff_values = [- y_value for y_value in diff_values] 
    
    # Create the waterfall chart
    fig = go.Figure(go.Waterfall(
        name="Waterfall Analysis",
        x=["All Customers"] + x_labels,
        y=[total_customers] + diff_values + [remaining_customers],  # Starting bar and placeholders
        measure=["absolute"] + ["relative"] * len(customers_after_filters) + ["absolute"],  # Absolute start, relative drops, absolute end
        text=[f"{val:,}" for val in [total_customers] + diff_values],  # Add text annotations
        decreasing={"marker": {"color": "red"}},
        connector={"line": {"color": "black", "dash": "dot"}},
        textposition = ["inside"] + ["outside"] * len(diff_values) + ["outside"]  # Use "outside" for diffs
    ))
    
    # Update layout
    fig.update_layout(
        title="Customer Reduction Waterfall Chart",
        xaxis_title="Filters (Name and SQL Condition)",
        yaxis_title="Number of Customers",
        showlegend=False,
        height=800
    )
    
    return fig

create_waterfall_chart(CND[:5], *get_waterfall_sequence(filter_cols[:5], tablename = "waterfall_base"))

## Create Interactive Widget for waterfall analyis

This code cell creates an interactive widget for generating waterfall charts based on selected filters. The waterfall_widget function utilizes IPywidgets and Plotly to create a user interface where users can select filters and view corresponding waterfall charts. The function dynamically updates the chart based on the selected filters.

In [ ]:
import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.display import display

def waterfall_widget(this_CND, tablename="waterfall_base"):
    # Extract allowed filter names from CND
    allowed_tags = [c[0] for c in this_CND]

    # Create a TagsInput widget for selecting filters in order, ensuring no duplicates
    filter_tags = widgets.TagsInput(
        value=[],
        allowed_tags=allowed_tags,
        placeholder="Select filters in order...",
        description="Filters:",
        allow_duplicates=False  # Ensure no duplicates are allowed
    )

    # Create a button to trigger the plot function
    plot_button = widgets.Button(
        description="Plot Waterfall",
        button_style="primary",  # Styles the button
        tooltip="Click to plot the waterfall chart",
        icon="bar-chart"  # Optional icon
    )

    # Placeholder for a Plotly figure, initially empty
    fig_placeholder = widgets.Output()

    # Function to be called when the button is clicked
    def on_plot_button_click(b):
        with fig_placeholder:
            fig_placeholder.clear_output()  # Clear previous plot output

            # Debug: Check if the function is triggered
            #print("Button clicked!")  
            
            # Read the selected filters from the TagsInput
            selected_filters = filter_tags.value
            #print("Selected filters:", selected_filters)  # Debug: Print selected filters

            if not selected_filters:
                #print("No filters selected!")
                return

            # Get the waterfall sequence using the selected filters
            total_customers, customers_after_filters = get_waterfall_sequence(selected_filters, tablename)

            # Reduce this_CND to only include selected filters, in the specified order
            reduced_CND = [c for c in this_CND if c[0] in selected_filters]
            reduced_CND = sorted(reduced_CND, key=lambda x: selected_filters.index(x[0]))

            # Create a new waterfall chart
            plot = create_waterfall_chart(reduced_CND, total_customers, customers_after_filters)
            
            # Display the plot using a standard Figure (non-widget)
            fig = go.Figure(plot)
            display(fig)

    # Link the button click event to the function
    plot_button.on_click(on_plot_button_click)

    # Display the widgets and the figure placeholder
    display(filter_tags, plot_button, fig_placeholder)


<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
waterfall_widget(___, tablename=___)  # Call the function with appropriate arguments

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
You need to provide the correct arguments for the `waterfall_widget` function. The first argument is likely a variable or data structure, and the second is a string representing the table name.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
waterfall_widget(CND, tablename="waterfall_base")
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

# Clustering as an Alternative Technique to Random Sampling for Selection of E-Mail Adresses


Creates a DataFrame object named DF_master by reading the "master_table" table from the "demo_user" schema.

In [ ]:
DF_master = tdml.DataFrame(tdml.in_schema("demo_user","master_table"))

## transform the master dataset into numeric columns, then PCA

Performs data preprocessing steps on the DF_master DataFrame, excluding the columns "Customer_Id" and "Master_Bank_Customer_Id", and then applies Principal Component Analysis (PCA) with 10 components. Fits a Pipeline object to the preprocessed data.

In [ ]:
import tdprepview

from tdprepview import Pipeline

steps = eval(tdprepview.auto_code(DF_master, non_feature_cols=["Customer_Id","Master_Bank_Customer_Id"]))

steps.append(  
    ({'columns_exclude': ['Customer_Id', 'Master_Bank_Customer_Id']}, 
				 tdprepview.PCA(n_components=10)))
print(steps)

pl = Pipeline(steps)

pl.fit(DF_master)

Plots a Sankey diagram based on the data transformation steps performed by the Pipeline object pl.

In [ ]:
pl.plot_sankey()

Transforms the DF_master DataFrame using the Pipeline object pl and displays the SQL query generated for the transformation. Prints the data types of the transformed DataFrame.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
DF_master_transformed = pl.transform(___)  # Transform the DataFrame using the appropriate function
prettyprint_sql(DF_master_transformed.show_query())  # Pretty print the SQL query of the transformed DataFrame

DF_master_transformed.___  # Access the teradata data types of the transformed DataFrame

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
First, ensure you are transforming the correct DataFrame using the `pl.transform` function. Then, use the `show_query` method to retrieve the SQL query from the transformed DataFrame. Finally, access the data types of the transformed DataFrame using the `tdtypes` attribute.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
DF_master_transformed = pl.transform(DF_master)
prettyprint_sql(DF_master_transformed.show_query())

DF_master_transformed.tdtypes
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

Drops the column "Master_Bank_Customer_Id" from the DF_master_transformed DataFrame, then saves the DataFrame as a table named "cluster_input" in the database with the primary index set to "Customer_Id". Creates a new DataFrame object DF_input_clustering by reading the "cluster_input" table.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
(DF_master_transformed
     .drop(columns=["Master_Bank_Customer_Id"])
     .to_sql("cluster_input", if_exists = "replace", primary_index = ___))  # Define primary index

DF_input_clustering = tdml.DataFrame(___)  # Use the table name from above

DF_input_clustering

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
Identify the column you want to drop from the DataFrame. Then, specify the name of the SQL table you want to create and the primary index for that table. Finally, ensure you use the same table name when creating the `tdml.DataFrame`.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
(DF_master_transformed
     .drop(columns=["Master_Bank_Customer_Id"])
     .to_sql("cluster_input", if_exists = "replace", primary_index = "Customer_Id"))

DF_input_clustering = tdml.DataFrame("cluster_input")

DF_input_clustering
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## Fit Cluster and assign Cluster to rows

This code cell prints the columns of the DataFrame DF_input_clustering.

In [ ]:
print(DF_input_clustering.columns)

This code cell performs KMeans clustering on the data in DF_input_clustering using the specified columns as features. It creates a KMeans object with 50 clusters and prints the result.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
KMeans_obj = tdml.KMeans(data = DF_input_clustering,  # Provide the input data for clustering
                        id_column = ___,  # Specify the ID column
                         target_columns = ___,  # List the target columns for clustering
                         num_clusters = 50,  # Define the number of clusters
                         seed=42,  # Set the random seed for reproducibility
                         initialcentroids_method= 'KMEANS++',  # Choose the method for initial centroids
                     volatile  = True  # Set volatility to True or False
                        )

KMeans_obj.result

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
Ensure you understand the parameters required for initializing a KMeans object. You need to specify the data, ID column, target columns, number of clusters, random seed, method for initial centroids, and whether the operation is volatile. The target columns are those columns used for calculating the cluster model
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
KMeans_obj = tdml.KMeans(data = DF_input_clustering, 
                        id_column = "Customer_Id",
                         target_columns = ['pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5', 'pc_6', 'pc_7', 'pc_8', 'pc_9', 'pc_10'],
                         num_clusters = 50,
                         seed=42,
                         initialcentroids_method= 'KMEANS++',
                     volatile  = True
                        )

KMeans_obj.result
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

This code cell predicts the cluster assignments for the data in DF_input_clustering using the KMeans clustering model created in the previous cell. It prints the shape of the resulting DataFrame and displays the DataFrame with cluster assignments.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
Kmeans_pred = tdml.KMeansPredict(
    data = ___,  # Provide the input data for clustering, same as for fitting the model.
    object = ___,  # Use the result from the KMeans object
    volatile = True 
)

DF_clusterassignment = Kmeans_pred.result

print(DF_clusterassignment.shape)  # Print the shape of the result

DF_clusterassignment

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">
Ensure you pass the correct data and KMeans object to the `KMeansPredict` function. The `volatile` parameter is a boolean, and you need to print the shape of the DataFrame using the appropriate attribute.
  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
Kmeans_pred = tdml.KMeansPredict(
    data = DF_input_clustering,
      object=KMeans_obj.result,
       volatile = True
              )

DF_clusterassignment = Kmeans_pred.result

print(DF_clusterassignment.shape)

DF_clusterassignment
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

## Random Samples from each Cluster

This code cell retrieves the name of the cluster assignment table, creates a DataFrame with a sample of 5 rows per cluster, sorts the sample by cluster ID and customer ID, and then groups the sample by cluster ID and counts the number of rows in each cluster.

<br><br><div style=" border-radius: 15px 15px 0 0; background: #a8d5e2; padding: 10px;"></div>

In [ ]:
cluster_table_name = DF_clusterassignment.db_object_name  # Access the db_object_name attribute
print(cluster_table_name)

DF_sample = tdml.DataFrame.from_query(f"""
SELECT *
FROM {cluster_table_name}
QUALIFY ROW_NUMBER() OVER (PARTITION BY ___ ORDER BY HASHAMP(HASHBUCKET(HASHROW(Customer_Id || 'myseed1')))) <= 5;
""")

DF_sample.sort(["td_clusterid_kmeans", "Customer_Id"])  # Sort by cluster ID and Customer_Id

DF_sample.groupby("td_clusterid_kmeans").count()  # Group by cluster ID and count

<details>
  <summary style="font-weight:bold; color:#9b870c;">Hint</summary>
  <div style="background-color:#fff9db; padding:10px; border-radius:5px; margin-top:5px;">

 The `PARTITION BY` clause in SQL is used to define how the data is divided into partitions. Use the column that represents the cluster ID.

  </div>
</details>

<details>
  <summary style="font-weight:bold; color:#0c9b3b;">Solution</summary>
  <div style="background-color:#dbffdb; padding:10px; border-radius:5px; margin-top:5px;">

```python
cluster_table_name = DF_clusterassignment.db_object_name
print(cluster_table_name)

DF_sample = tdml.DataFrame.from_query(f"""
SELECT *
FROM {cluster_table_name}
QUALIFY ROW_NUMBER() OVER (PARTITION BY td_clusterid_kmeans ORDER BY HASHAMP(HASHBUCKET(HASHROW(Customer_Id || 'myseed1')))) <= 5;
""")

DF_sample.sort(["td_clusterid_kmeans","Customer_Id"])

DF_sample.groupby("td_clusterid_kmeans").count()
```
  </div>
</details>

<div style=" border-radius: 0 0 15px 15px ; background: #a8d5e2; padding: 10px;"></div><br><br>

# clean up db & remove context

This code cell iterates through a list of database objects and attempts to drop tables and views with those names. It prints a message for each object deleted.

In [ ]:
db_obj_list = [
 'Account_Customer_Map',
 'Account_Dim',
 'Balance_Fact',
 'Customer_Details',
 'Customer_Dim',
 'Interaction_Fact',
 'Master_Table',
 'Transaction_Fact',
 'cluster_input',
 'customer_BEZH_EN',
 'master_filter',
 'quality_customerdetails',
 'waterfall_base'
]

for torv in db_obj_list:
    try:
        tdml.db_drop_table(torv)
        print(f"Table {torv} deleted.")
    except:
        pass
    try:
        tdml.db_drop_view(torv)
        print(f"View {torv} deleted.")
    except:
        pass

This code removes the current active context in Teradata.

In [ ]:
tdml.remove_context()